In [ ]:
## Exercise 14
## Team Dzjio Skripting
## Gersom Zomer & Yannick Mijnheer
## 26 January 2017

In [2]:
## import packages
from twython import Twython
from osgeo import ogr,osr
import json
import datetime
import csv
import folium
import os

## access twitter API 
APP_KEY = 'W5NxkTOBUzqxOwVMue0zhBpCI'
APP_SECRET = 'kydR7oDhbMvdZ7LkU4eES47ogdlXWDccgwxqxwrPjJ9TwfDjw1'
OAUTH_TOKEN = '759844343050035200-T3OjfWEWLNNyXlYBRXp58NS3mV8uyyJ'
OAUTH_TOKEN_SECRET = 'UgkO3nQKB9Yoc2yI1okRZ1sxGf9ekguNFWICmZsaxIz8y'

## initiate Twython object 
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [19]:
## Define a function to search for tweets:
## This is the main function, in which you can specify your twitter-search.
## This function does not need any input, since everything is set by default. But you
## have the option to change the hashtag-search, count, radius, latitude and longitude.

def search_twitter(hashtag='Amsterdam', count=25, radius='25km', lat=51.9700000,lon=5.6666700):
    search = twitter.search(q='#%s'%(hashtag),geocode='%.6f,%.6f,%s'%(lat,lon,radius), count=count)
    user  = []
    foll  = []
    lati  = []
    longi = []
    for tweet in search["statuses"]:
        coordinates = tweet['coordinates']
        if coordinates!= None:
            tweettext = tweet['text'] 
            user  = user  + [tweet['user']['screen_name']] # username
            foll  = foll  + [tweet['user']['followers_count']] # number of followers
            lati  = lati  + [coordinates['coordinates'][1]] # latitude
            longi = longi + [coordinates['coordinates'][0]] # longitude
    result = zip(user, foll, lati, longi)
    return result


## Define a function to save the results to csv-file:
## This function is created to serve the output of search_twitter, and 
## by default save it as 'Ex14_twitter_results.csv'.

def csv_twitter(inputdata, output = 'Ex14_twitter_results.csv'):
    with open(output,'wb') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['Username','Followers','Latitude','Longitude'])
        for row in inputdata:
            csv_out.writerow(row)

            
## Define a function to create a shapefile from the search_twitter results:
## The input for this function consists of: 
## (input data(list of tuples), output shapefile name, output layer name)

def shp_twitter(datalist, outshape = 'Ex14_tweets.shp', outlayer = 'Ex14_layer'): 
    ## Set ESRI shapefile driver
    drv = ogr.GetDriverByName("ESRI Shapefile")
    if drv is None:
        print "%s driver not available.\n" % driverName
        pass
    
    ## Create a shape file
    ds = drv.CreateDataSource(outshape)
    
    # Set spatial reference
    spatialReference = osr.SpatialReference()
    spatialReference.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    
    ## Create a layer
    layer = ds.CreateLayer(outlayer, spatialReference, ogr.wkbPoint)
    layerDefinition = layer.GetLayerDefn()
    
    ## Create points
    for i in range(len(datalist)):
        exec("point%d = ogr.Geometry(ogr.wkbPoint)" % (i+1))
        ## SetPoint(self, int point, double x, double y, double z = 0)
        exec("point%d.SetPoint(0,%.6f,%.6f)" % (i+1,datalist[i][3],datalist[i][2]))
        ## Create features
        exec("feature%d = ogr.Feature(layerDefinition)" % (i+1))
        ## Add the points to the features
        exec("feature%d.SetGeometry(point%d)" % (i+1,i+1))
        ## Store the feature in a layer
        exec("layer.CreateFeature(feature%d)" % (i+1))
    
    ## The shapefile is updated with all object structures 
    ## when the script is destroyed:
    ds.Destroy()

In [20]:
## test the functions:

Coffee_NewYork = search_twitter(hashtag='coffee', count=200, lat = 40.730610, lon = -73.935242) 
print(Coffee_NewYork)

csv_twitter(Coffee_NewYork, 'Tweets_Coffee_NewYork.csv')
shp_twitter(Coffee_NewYork, outshape = 'Tweets_Coffee_NewYork.shp')
